1. Visión del proyecto (para explicarlo en clase/entrevista)

“Construí un generador de datos sintéticos que usa modelos de lenguaje open-source (Llama 3.1 / 3.2 y Gemma 2) para crear datasets tabulares a partir de descripciones de negocio. El usuario define el dominio (ej. ventas, banca, salud), el tamaño del dataset y las reglas; el modelo genera datos en formato CSV. Encima de esto construí una interfaz con Gradio para que cualquier usuario pueda usarlo sin escribir código.”

Casos de uso a mencionar:

- Probar pipelines de datos cuando no hay datos reales disponibles.
- Crear datos despersonalizados para demos / prototipos.
- Augmentación de datos para ejercicios de ML.

2. Tecnologías a usar

- Modelos LLM (texto-texto):
- meta-llama/Llama-3.1-8B-Instruct 
- meta-llama/Llama-3.2-3B-Instruct 
- google/gemma-2-9b-it 

Librerías Python:

- transformers o huggingface_hub (para llamar al modelo).
- pandas (para construir el DataFrame).

gradio (UI).

- Auth Hugging Face con token. 

3. Estructura sugerida del notebook Project3-week3.ipynb

- Título + descripción del proyecto (Markdown).
- Instalación / imports.
- Configuración de modelos Hugging Face.
- Definición de esquemas de datasets (plantillas de negocio).
- Función generadora usando LLM.
- Conversión a pandas.DataFrame + validaciones básicas.
- Interfaz Gradio.

Pruebas y ejemplos de uso.

### Imports e instalación

In [ ]:
!pip install -q transformers huggingface_hub gradio pandas BitsAndBytesConfig dotenv openai torch

In [ ]:
import os
import re
import pandas as pd
from huggingface_hub import InferenceClient, login
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI


### Login Hugging face and OpenAI

In [ ]:
# Load environment variables
load_dotenv(dotenv_path='/workspace/.env', override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
hf_token = os.getenv('HUGGINGFACE_API_KEY')

# Initialize OpenAI Client
if openai_api_key:
    openai_client = OpenAI(api_key=openai_api_key)
    print(" OpenAI Client Initialized")
else:
    print(" OpenAI API Key not found")

# Login to Hugging Face
if hf_token:
    if hf_token.startswith('Bearer '):
        hf_token = hf_token.replace('Bearer ', '')
    login(hf_token.strip())
    print(" Logged into Hugging Face")
else:
    print(" Hugging Face Token not found")

### Configuración de modelos

In [ ]:
# Define model identifiers
LLAMA_3_1 = "meta-llama/Llama-3.1-8B-Instruct"
LLAMA_3_2 = "meta-llama/Llama-3.2-3B-Instruct"
PHI4 = "microsoft/Phi-3-mini-4k-instruct"
GEMMA3 = "google/gemma-3-1b-it"

### Cache Management

To keep our workspace organized, we will define specific directories for each model. This prevents models from filling up the default cache partition and allows for easier management.

In [ ]:
# Base Hugging Face cache directory
hf_cache_base = os.getenv('HF_HOME', '/root/.cache/huggingface')

# Define specific cache directories for each model
model_cache_llama = os.path.join(hf_cache_base, 'models', 'llama_3_1_8b')
model_cache_llama = os.path.join(hf_cache_base, 'models', 'llama_3_2_8b')
model_cache_phi = os.path.join(hf_cache_base, 'models', 'phi_3_mini')
model_cache_gemma = os.path.join(hf_cache_base, 'models', 'gemma_3_4b')

# Create directories if they don't exist
os.makedirs(model_cache_llama, exist_ok=True)
os.makedirs(model_cache_phi = os.path.join(hf_cache_base, 'models', 'phi_3_mini')
, exist_ok=True)
os.makedirs(model_cache_gemma, exist_ok=True)

print(f"Llama Cache: {model_cache_llama}")
print(f"Phi-3 Cache: {model_cache_phi}")
print(f"Gemma Cache: {model_cache_gemma}")

### Quantization Theory & Configuration

In [ ]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)